# Autograd

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adamelliotfields/ml/blob/main/notebooks/autograd.ipynb)
[![Render nbviewer](https://img.shields.io/badge/render-nbviewer-f37726)](https://nbviewer.org/github/adamelliotfields/ml/blob/main/notebooks/autograd.ipynb)

This is a neural network I put together from scratch.

The key feature is _automatic differentiation_, which we get from the [Autograd](https://github.com/HIPS/autograd) package. This enables us to compute the gradients of our loss function.

The interface is the same as NumPy with the additional `grad` function. The benefit of Autograd over the newer JAX (and PyTorch) is that it's pure Python, so it can be installed anywhere.

In [1]:
import autograd.numpy as np
import pandas as pd
from autograd import grad
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from tqdm import tqdm

np.random.seed(42)

This neural network classifies Iris flowers using the famous [Iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set) into three species: setosa, versicolor, and virginica. Species is the target variable.

The features (independent variables) are 4 measurements of the flowers: sepal length, sepal width, petal length, and petal width.

The architecture is a simple feedforward neural network also known as a multilayer perceptron (MLP). It has 4 input neurons, 3 output neurons, and 1 hidden layer with 8 neurons. Unlike more advanced architectures, a feedforward net has no loops (recurrent connections) or skips (residual connections); it's just a sequence of layers.

In [2]:
# hyperparameters
# play around with these!
LAYER_SIZES = [4, 8, 3]
LEARNING_RATE = 1e-2  # 0.01
EPOCHS = 50

# column names
columns = ["sepal_length", "sepal_width", "petal_length", "petal_width"]

# load and convert to dataframe
iris = load_iris()
iris_df = pd.DataFrame(iris["data"], columns=columns)

# one-hot encode species
species = iris["target_names"]
iris_df["species"] = species[iris["target"]]
iris_df = pd.get_dummies(iris_df, columns=["species"])

# train-test split
X = iris_df[columns].values
y = iris_df.iloc[:, -3:].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
iris_df.sample(5, random_state=42)

,sepal_length,sepal_width,petal_length,petal_width,species_setosa,species_versicolor,species_virginica
73,6.1,2.8,4.7,1.2,0,1,0
18,5.7,3.8,1.7,0.3,1,0,0
118,7.7,2.6,6.9,2.3,0,0,1
78,6.0,2.9,4.5,1.5,0,1,0
76,6.8,2.8,4.8,1.4,0,1,0


First we need to initialize the parameters with random numbers to start (`init_params`).

The activation function (`relu`) simply returns the input if it's positive, and 0 otherwise.

The `logsumexp` function is a _numerically stable_ way to compute the log of the sum of the exponentials for the `softmax` activation function in the output layer. When dealing with really small or really big numbers, we can run into underflow or overflow errors. This means that tiny numbers are rounded to zero and huge numbers are rounded to infinity. In machine learning, these rounding errors could lead to inaccurate predictions.

The `predict` function returns _logits_, which we convert to probabilities in the output layer. Logits are raw (not normalized) numbers generated by the model. They are the input to the softmax function, which returns a distribution of probabilities over the classes.

The `loss` function is _category cross-entropy_ (CCE). This is a measure of the dissimilarity between the true distribution and the predicted distribution. The loss function is _minimized_ when the predicted distribution is close to the true distribution. Worth mentioning that in PyTorch, CCE includes softmax, so raw logits should be passed.

> Note that JAX provides `logsumexp`; Autograd does not. We cannot use SciPy's because it wouldn't be differentiable, so we have to write our own with `autograd.numpy`.

In [3]:
# initialize params
def init_params(sizes):
    params = []  # weight/bias pairs
    for m, n in zip(sizes[:-1], sizes[1:]):
        weight = 1e-2 * np.random.randn(n, m)
        bias = 1e-2 * np.random.randn(n)
        params.append((weight, bias))
    return params


# relu activation in hidden layers
def relu(x):
    return np.maximum(0, x)


# logsumexp for numerical stability
# need to reshape because we can't use `keepdims` for `max` in autograd.numpy
def logsumexp(logits):
    max_logits = np.max(logits, axis=1)
    sum_exp = np.sum(np.exp(logits - max_logits[:, None]), axis=1)
    log_sum_exp = max_logits + np.log(sum_exp)
    return log_sum_exp[:, None]


# softmax activation in output layer
def softmax(logits):
    shifted_logits = logits - logsumexp(logits)
    return np.exp(shifted_logits)


# predict function
def predict(params, example):
    # ensure example is 2D, even if single example
    if example.ndim == 1:
        example = np.expand_dims(example, axis=0)  # same as `[None, :]`

    # loop over all layers except output
    activations = example
    for weight, bias in params[:-1]:
        # dot product of the weights and the activations of the previous layer plus bias
        # (transpose to ensure dimensions align)
        outputs = np.dot(weight, activations.T).T + bias
        activations = relu(outputs)

    # extract the parameters for the final layer
    final_weight, final_bias = params[-1]
    logits = np.dot(final_weight, activations.T).T + final_bias

    # subtract the log of the sum of the exponentiated logits for numerical stability
    return logits - logsumexp(logits)


# accuracy function
def accuracy(params, example, target):
    logits = predict(params, example)
    preds = np.argmax(logits, axis=1)
    targets = np.argmax(target, axis=1)
    return np.mean(preds == targets)


# loss function
def loss(params, example, target):
    logits = predict(params, example)
    probs = softmax(logits)
    return -np.mean(np.sum(target * np.log(probs), axis=1))  # categorical cross-entropy


# single step of gradient descent for training loop
def update(params, x, y, learning_rate):
    grads = grad(loss)(params, x, y)
    new_params = []
    for (weight, bias), (weight_grad, bias_grad) in zip(params, grads):
        new_weight = weight - learning_rate * weight_grad
        new_bias = bias - learning_rate * bias_grad
        new_params.append((new_weight, new_bias))
    return new_params

In [4]:
# init network
model = init_params(LAYER_SIZES)

# train!
prog_bar = tqdm(range(EPOCHS))
for epoch in prog_bar:
    for example, target in zip(X_train, y_train):
        example_2d = np.expand_dims(example, axis=0)
        target_2d = np.expand_dims(target, axis=0)
        model = update(model, example_2d, target_2d, LEARNING_RATE)
    acc = accuracy(model, X_test, y_test)
    lss = loss(model, X_test, y_test)
    prog_bar.set_postfix(accuracy=f"{acc:.3f}", loss=f"{lss:.3f}")

100%|█████████████████████████████████████| 50/50 [00:07<00:00,  6.30it/s, accuracy=0.967, loss=0.109]


The trained model looks like this:

```json
[
  [
    [
      [-5.34416454e-03, -1.49505387e-02, -7.89258329e-03, 7.43711284e-03],
      [-1.52876246e-01, -4.83584651e-01, 1.25840621e+00, 7.66275274e-01],
      [-1.12059472e+00, -1.17667945e+00, 1.99320102e+00, 1.56054733e+00],
      [-4.03155807e-03, 6.01119070e-03, -2.81482396e-03, -3.71979647e-03],
      [ 8.77257997e-03, -5.69623799e-03, -2.75949741e-02, 3.27996260e-03],
      [-3.95516452e-03, -2.89136856e-03, 4.52936327e-03, -1.66060908e-03],
      [ 2.14938830e-03, -2.02231493e-02, -9.43056808e-03, 1.40395874e-02],
      [-1.85508045e-04, -1.67350462e-02, -1.07253183e-02, -9.92586179e-03]
    ],
    [1.02347683e-03, -1.31112511e-01, -5.66512209e-01, -1.49189067e-04, 3.99712755e-03, -2.59028645e-03, -5.74709208e-03, -4.21498220e-03]
  ], [
    [
      [3.39820964e-03, -1.55629341e+00, -6.26243719e-01, -1.13993816e-02, -6.76569892e-03, 7.73140856e-03, -8.01827843e-03, 1 38401572e-02],
      [1.40520531e-02, 8.38914200e-01, -1.04577923e+00, 7.19771668e-04, -5.42906466e-03,  9.23162608e-03, 1.70660495e-02, 8.73589416e-03],
      [9.14434574e-05, 7.27573270e-01, 1.67738038e+00, -1.22781017e-02, 4.87043802e-03, -9.14690931e-03, 6.20548216e-03, -1.60937377e-03]
    ],
    [2.75982171, -0.11198307, -2.66414385]
  ]
]
```

So, `model[0][0]` is a 2D array of shape `(8, 4)`, meaning there are 8 arrays of length 4. There are 4 input features corresponding to the 4 measurements of the iris flowers. There are 8 neurons in the first "hidden" layer.

The next item is a 1D array of shape `(8,)` (length of 8). These are the biases of the 8 neurons in the first layer.

Biases are like the intercept in a linear equation. If you were predicting the price of a house, in a linear model the bias would be the baseline price before taking into account the number of bedrooms, bathrooms, etc.

The output layer has 3 neurons corresponding to the 3 species of iris flowers. The weights are a 2D array of shape `(3, 8)`: 3 arrays of length 8, or 3 neurons with 8 inputs each. The biases are a 1D array of shape `(3,)`.